In [1]:
from PAc_optimal_bounds import *

In [2]:
import sys, os

def get_env_type() -> str:
    '''
    Get the environment type where the code is running.

    Returns:
    - 'kaggle' if running on Kaggle
    - 'google.colab' if running on Google Colab
    - 'local' if running on local environment
    '''
    if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        return 'kaggle'
    elif 'google.colab' in sys.modules:
        if 'COLAB_TPU_ADDR' in os.environ:  # Google Colab w/ TPU
            # Connect to TPU
            import tensorflow
            tpu = tensorflow.distribute.cluster_resolver.TPUClusterResolver()
            tensorflow.config.experimental_connect_to_cluster(tpu)
            tensorflow.tpu.experimental.initialize_tpu_system(tpu)
        # Connect to Drive
        from google.colab import drive
        drive.mount('/content/drive')

        # load the custom PAc Optimal Bounds module
        %cd "/content/drive/MyDrive/Colab Notebooks/PAc"

        return 'google.colab'
    else:   # Running on local environment

        # load the custom PAc Optimal Bounds module
        %cd "C:/Users/igorv/OneDrive/DL/PAc"

        return 'local'


In [3]:
print_versions_and_GPU()

match get_env_type():
    case 'kaggle':
        raise ValueError('This notebook is not designed to run on Kaggle.')
    case 'google.colab':
        data_path = '/content/drive/MyDrive/data/PAc/bounds/'
    case 'local':
        data_path = "C:/Users/igorv/OneDrive/DL/PAc/"
    case _:
        raise ValueError(f'Unknown environment type: {get_env_type()}')

print(f'Running on {get_env_type()}')

Python: 3.11.5
TensorFlow: 2.16.1
Keras: 3.1.1
Scikit-learn: 1.2.2
C:\Users\igorv\OneDrive\DL\PAc
C:\Users\igorv\OneDrive\DL\PAc
Running on local


In [4]:
import tensorflow as tf

approx_silu = PAc_Approximator(
    func=lambda x: tf.nn.silu(x).numpy(),
    func_low=lambda x: 0,
    func_high=lambda x: x)


In [6]:
for sz in [16, 32, 64, 128, 256, 512, 1024, 2048, 4096]:
    multi_opt_max_runner(data_path, f'silu_emax_{sz}', approx_silu.approx_max_e_swarm, approx_silu, n=sz, num_runs=5, max_iter=1000, sig_digits=4, patience=30)

2024-06-03 16:58:20.121004 Starting silu_emax_16, 5 runs
2024-06-03 16:58:20.126999 Starting optimization 1000 2.78464e-01 ...2.59251e-01 .2.01238e-01 .................1.89317e-01 ........................1.85445e-01 ......1.83420e-01 ..............1.82712e-01 ........1.81841e-01 ................1.81727e-01 ............................. - Early stopping: 0.18172677040098995 @ [-2.7001852   2.58714494]
2024-06-03 16:58:38.378355 Starting optimization 1000 3.22322e-01 2.96196e-01 ..2.78464e-01 2.64679e-01 .2.34491e-01 ..2.28152e-01 .......2.18094e-01 ............2.14099e-01 2.09203e-01 ..................2.00538e-01 ............................. - Early stopping: 0.20053840876739448 @ [-2.52123248  3.31614613]
2024-06-03 16:58:50.119128 Starting optimization 1000 3.56847e-01 ..3.04345e-01 2.78464e-01 2.29103e-01 ..2.20513e-01 ......2.02609e-01 .....1.94922e-01 ..................1.85684e-01 .........1.83466e-01 ......1.82705e-01 .....1.82268e-01 .......1.81480e-01 .....................1.814

In [ ]:
engine = PSOEngine2D()
engine.optimize(approx_silu.approx_max_e_swarm, max_iter=1000, sig_digits=5, patience=50, n=256)

In [ ]:
from pyswarms.utils.plotters import plot_cost_history, plot_contour, plot_surface
import matplotlib.pyplot as plt

plot_cost_history(cost_history=engine.optimizer.cost_history)

In [ ]:
best_x_min, best_x_max = engine.best_pos[0], engine.best_pos[1]


In [ ]:
plot_silu_256_10_10 = Error3D(data_path, "silu_emax_256_10_10", "PAc-SiLU (256)", best_x_min, best_x_max, approx_silu, 256, -10., 0., 0., 10., 50)

In [ ]:
plot_silu_256_10_10.plot_combined_error()
plot_silu_256_10_10.plot_max_error()
plot_silu_256_10_10.plot_auc()

In [ ]:
plot_silu_256_2_2 = Error3D(data_path, "silu_emax_256_2_2", "PAc-SiLU (256)", best_x_min, best_x_max, approx_silu, 256, -6.5, -4.5, 4.5, 6.5, 50)

In [ ]:
plot_silu_256_2_2.plot_combined_error()
plot_silu_256_2_2.plot_max_error()
plot_silu_256_2_2.plot_auc()

In [ ]:
plot_silu_256_05_05 = Error3D(data_path, "silu_emax_256_05_05", "PAc-SiLU (256)", best_x_min, best_x_max, approx_silu, 256, -5.75, -5.25, 5.25, 5.75, 50)

In [ ]:
plot_silu_256_05_05.plot_combined_error()
plot_silu_256_05_05.plot_max_error()
plot_silu_256_05_05.plot_auc()